In [1]:
import os
from PIL import Image
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import matplotlib.pyplot as plt

device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Định nghĩa lớp Dataset tùy chỉnh
class CatsDogsDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_paths = [os.path.join(root_dir, fname) for fname in os.listdir(root_dir) if fname.endswith('.jpg')]
        self.labels = [1 if 'dog' in fname else 0 for fname in os.listdir(root_dir) if fname.endswith('.jpg')]

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path)
        if self.transform:
            image = self.transform(image)
        label = self.labels[idx]
        return image, label

# Tiền xử lý dữ liệu
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
])

# Tải dữ liệu
train_data = CatsDogsDataset(root_dir='dogs-vs-cats/train', transform=transform)
test_data = CatsDogsDataset(root_dir='dogs-vs-cats/test1', transform=transform)

# Cập nhật batch_size = 64
train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
test_loader = DataLoader(test_data, batch_size=64, shuffle=True)

# Xây dựng mô hình CNN
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64 * 32 * 32, 512)
        self.fc2 = nn.Linear(512, 2)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = x.view(-1, 64 * 32 * 32)
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x

model = SimpleCNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

# Huấn luyện mô hình
def train_model(model, train_loader, test_loader, criterion, optimizer, num_epochs=25):
    train_losses = []
    train_accuracies = []
    val_losses = []
    val_accuracies = []

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0

        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        train_losses.append(running_loss / len(train_loader))
        train_accuracy = 100 * correct / total
        train_accuracies.append(train_accuracy)

        # Đánh giá trên tập kiểm tra
        model.eval()
        val_loss = 0.0
        correct = 0
        total = 0
        with torch.no_grad():
            for images, labels in test_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        val_losses.append(val_loss / len(test_loader))
        val_accuracy = 100 * correct / total
        val_accuracies.append(val_accuracy)

        print(f"Epoch [{epoch+1}/{num_epochs}], "
              f"Train Loss: {train_losses[-1]:.4f}, Train Acc: {train_accuracy:.2f}%, "
              f"Val Loss: {val_losses[-1]:.4f}, Val Acc: {val_accuracy:.2f}%")

    return train_losses, train_accuracies, val_losses, val_accuracies

# Chạy huấn luyện và đánh giá
train_losses, train_accuracies, val_losses, val_accuracies = train_model(
    model, train_loader, test_loader, criterion, optimizer, num_epochs=25
)

Epoch [1/25], Train Loss: 0.6658, Train Acc: 58.71%, Val Loss: 0.7528, Val Acc: 54.30%
Epoch [2/25], Train Loss: 0.5795, Train Acc: 69.43%, Val Loss: 0.8440, Val Acc: 61.34%
Epoch [3/25], Train Loss: 0.5199, Train Acc: 73.93%, Val Loss: 0.8187, Val Acc: 63.47%
Epoch [4/25], Train Loss: 0.4566, Train Acc: 78.36%, Val Loss: 1.1461, Val Acc: 52.59%
Epoch [5/25], Train Loss: 0.4040, Train Acc: 81.66%, Val Loss: 1.2045, Val Acc: 54.50%
Epoch [6/25], Train Loss: 0.3363, Train Acc: 85.37%, Val Loss: 1.6603, Val Acc: 46.73%
Epoch [7/25], Train Loss: 0.2519, Train Acc: 89.56%, Val Loss: 1.8981, Val Acc: 45.99%
Epoch [8/25], Train Loss: 0.1611, Train Acc: 93.80%, Val Loss: 1.9227, Val Acc: 54.87%
Epoch [9/25], Train Loss: 0.0870, Train Acc: 97.04%, Val Loss: 2.5721, Val Acc: 52.71%


In [ ]:
# Vẽ biểu đồ
epochs = np.arange(1,26)

plt.figure(figsize=(15, 7))

plt.subplot(1, 2, 1)
plt.plot(epochs, train_losses, 'bo-', label='Training loss')
plt.plot(epochs, val_losses, 'ro-', label='Validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(epochs, train_accuracies, 'bo-', label='Training accuracy')
plt.plot(epochs, val_accuracies, 'ro-', label='Validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy (%)')
plt.title('Training and Validation Accuracy')
plt.legend()

plt.tight_layout()
plt.show()